In [97]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
from sklearn.base import clone
from sklearn.model_selection import train_test_split
from itertools import combinations
from sklearn.linear_model import LogisticRegression

In [98]:
class sbs():
    def __init__(self, k_features,  estimator, test_size = 0.3, score = accuracy_score, random_state=0):
        self.estimator = clone(estimator)
        self.test_size = test_size
        self.score = accuracy_score
        self.random_state = random_state
        self.k_features = k_features
    def fit(self, X, Y):
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=self.random_state, stratify=Y, test_size=self.test_size)
        dim = X.shape[1]
        self.indices = tuple(range(dim))
        self.subsets = [self.indices]
        score = self.calc_score(X_train, X_test, Y_train, Y_test , self.indices)
        self.scores = [score]
        while dim > self.k_features:
            score = []
            subsets = []
            for i in combinations(self.indices, r = dim-1):
                acc = self.calc_score(X_train, X_test, Y_train, Y_test , i)
            score.append(acc)
            subsets.append(i)
            best = np.argmax(score)
            self.indices = subsets[best]
            self.subsets.append(self.indices)
            dim -=1 
            self.scores.append(score[best])

        self.k_feature_score = self.scores[-1]
        return self
    def transform(self, X):
        return X[:, self.inidces]
            

    def calc_score(self, X_train, X_test, Y_train, Y_test , indices):
        self.estimator.fit(X_train[:,indices], Y_train)
        predictions = self.estimator.predict(X_test[:,indices])
        score = accuracy_score(predictions, Y_test)
        return score

In [99]:
os.chdir("/Users/ishitajain/Documents/Machine_learning_and_coding/codes/dna_classification/gene_expression_dataset/archive/")

In [111]:
data = pd.read_csv("data_set_ALL_AML_train.csv")
test_data = pd.read_csv("data_set_ALL_AML_independent.csv")
labels = pd.read_csv("actual.csv")
data_train = [col for col in data.columns if "call" not in col]
data_test = [col for col in test_data.columns if "call" not in col]
#print(labels.head)
#print(len(labels))
#print(len(test_data))
print(data_train)

['Gene Description', 'Gene Accession Number', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '34', '35', '36', '37', '38', '28', '29', '30', '31', '32', '33']


In [112]:
X_train = data[data_train]
X_test = test_data[data_test]
x_train= X_train.drop(['Gene Description', 'Gene Accession Number'], axis=1)
x_test = X_test.drop(['Gene Description', 'Gene Accession Number'], axis =1)

####preparing the test data

Y_train = labels[labels['patient'] <= 38]
Y_test = labels[labels['patient'] > 38]
Y_train = Y_train['cancer']
Y_test = Y_test['cancer']

In [104]:
print(X_train.shape)
print(X_test.shape)

(7129, 38)
(7129, 34)


In [105]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, Y_train)
predictions = clf.predict(X_test)

ValueError: Number of labels=38 does not match number of samples=7129

In [26]:
acc = accuracy_score(predictions, Y_test)
print(acc)

0.6176470588235294


In [27]:
from sklearn.feature_selection import RFE
selector = RFE(clf, n_features_to_select=500, step=1)
selector.fit(X_train, Y_train)


RFE(estimator=DecisionTreeClassifier(random_state=0), n_features_to_select=500)

In [29]:
##feature_number = selector.n_features_
##print(feature_number)

500


In [32]:
##X_train_1 = selector.transform(X_train)
##X_test_1 = selector.transform(X_test)

In [108]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
pca = PCA(n_components=2)


In [113]:
X_train_scaled = sc.fit_transform(x_train)
X_test_scaled = sc.transform(x_test)
X_train_reduced = pca.fit_transform(x_train_scaled)
X_test_reduced = pca.transform(x_test_scaled)
#lr = LogisticRegression()
#lr.fit(X_train_reduced, Y_train)
print(pca.explained_variance_ratio_)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- 39
- 40
- 41
- 42
- 43
- ...
Feature names seen at fit time, yet now missing:
- 1
- 10
- 11
- 12
- 13
- ...


In [75]:
pred = lr.predict(X_test_reduced)
print(np.sum(Y_test == pred))
print(lr.coef_)

17
[[ 0.00131224 -0.00213848 -0.00011595  0.00034408  0.00164316]]


In [60]:
pca.components_

array([[-2.58603801e-04,  1.60119251e-04,  1.27803215e-04, ...,
         1.22295402e-04,  5.74702505e-03,  1.06625400e-04],
       [ 7.79391144e-05, -3.17385634e-04, -8.27458229e-04, ...,
         4.65544337e-04, -1.10301446e-02, -1.65373988e-04]])

In [86]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=85, bootstrap=True)
clf.fit(X_train_reduced, Y_train)
predict = clf.predict(X_test_reduced)
print(accuracy_score(predict, Y_test))

0.6176470588235294
